<a href="https://colab.research.google.com/github/azowz/Sign-language/blob/main/OCR_Hamed_project_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os
import tarfile
import shutil
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def find_dataset_tar():
    """Search for IIIT5K dataset tar file in common locations"""
    # First check current directory
    for filename in os.listdir("."):
        if "iiit5k" in filename.lower() and (filename.endswith(".tar") or filename.endswith(".tar.gz")):
            return filename

    # Then check sample_data directory (common in Colab)
    sample_data = "sample_data"
    if os.path.exists(sample_data):
        for filename in os.listdir(sample_data):
            if "iiit5k" in filename.lower() and (filename.endswith(".tar") or filename.endswith(".tar.gz")):
                return os.path.join(sample_data, filename)

    # If still not found, check for any tar file
    for filename in os.listdir("."):
        if filename.endswith(".tar") or filename.endswith(".tar.gz"):
            return filename

    # Final fallback: look for extracted dataset
    if os.path.exists("IIIT5K"):
        return "ALREADY_EXTRACTED"

    return None

def extract_and_verify_dataset():
    """Extract the IIIT5K dataset with flexible location handling"""
    # Try to find the dataset
    tar_path = find_dataset_tar()

    if not tar_path:
        print("="*50)
        print("ERROR: IIIT5K dataset not found!")
        print("Please upload the dataset using:")
        print("1. Google Drive: Mount Drive and specify path")
        print("2. Direct upload: Click 'Upload' in Colab file browser")
        print("3. Or download with: !wget http://cvit.iiit.ac.in/research/projects/cvit-projects/the-iiit-5k-word-dataset/IIIT5K-Word_V3.0.tar")
        print("="*50)
        return None

    # Create output directory
    output_dir = "iiit5k_dataset"
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

    # Handle already extracted dataset
    if tar_path == "ALREADY_EXTRACTED":
        print("✓ Found pre-extracted dataset in 'IIIT5K' folder")
        print("Copying to working directory...")
        for item in os.listdir("IIIT5K"):
            shutil.move(os.path.join("IIIT5K", item), output_dir)
        shutil.rmtree("IIIT5K")
        return verify_dataset_structure(output_dir)

    # Extract the tar file
    print(f"✓ Found dataset: {tar_path}")
    print(f"Extracting to {output_dir}...")
    try:
        if tar_path.endswith(".tar.gz"):
            with tarfile.open(tar_path, "r:gz") as tar:
                tar.extractall(output_dir)
        else:
            with tarfile.open(tar_path, "r") as tar:
                tar.extractall(output_dir)
        print("✓ Extraction completed successfully!")
    except Exception as e:
        print(f"✗ Error extracting file: {e}")
        return None

    return verify_dataset_structure(output_dir)

def verify_dataset_structure(output_dir):
    """Verify the dataset structure after extraction"""
    # Standard IIIT5K structure
    train_dir = None
    test_dir = None
    gt_file = None

    # Search for components
    for root, dirs, files in os.walk(output_dir):
        for dir_name in dirs:
            if "train" in dir_name.lower() and not any(x in dir_name.lower() for x in ["test", "val"]):
                train_dir = os.path.join(root, dir_name)
            if "test" in dir_name.lower() and not any(x in dir_name.lower() for x in ["train", "val"]):
                test_dir = os.path.join(root, dir_name)

        for file in files:
            if "gt" in file.lower() and file.endswith((".mat", ".txt")):
                gt_file = os.path.join(root, file)

    # Check image counts
    train_count = 0
    test_count = 0
    if train_dir and os.path.exists(train_dir):
        train_count = len([f for f in os.listdir(train_dir)
                          if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    if test_dir and os.path.exists(test_dir):
        test_count = len([f for f in os.listdir(test_dir)
                         if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

    # Create verification report
    report = {
        "base_dir": output_dir,
        "train_dir": train_dir,
        "test_dir": test_dir,
        "gt_file": gt_file,
        "train_count": train_count,
        "test_count": test_count,
        "valid": bool(train_dir and test_dir and gt_file and (train_count > 0 or test_count > 0))
    }

    # Print verification
    print("\n" + "="*50)
    print("DATASET VERIFICATION")
    print("="*50)
    print(f"Training images: {train_count} {'✓' if train_count > 0 else '✗'}")
    print(f"Test images: {test_count} {'✓' if test_count > 0 else '✗'}")
    print(f"Ground truth: {os.path.basename(gt_file) if gt_file else 'Not found'} {'✓' if gt_file else '✗'}")

    if report["valid"]:
        print("\n✓ Dataset verified successfully!")
        return report
    else:
        print("\n✗ Dataset verification failed. Common issues:")
        print("  - Incorrect dataset format (should be IIIT5K-Word_V3.0)")
        print("  - Missing ground truth files")
        print("  - Corrupted extraction")
        return None

if __name__ == "__main__":
    print("="*50)
    print("IIIT5K DATASET LOADER - SMART FIX")
    print("="*50)

    # Attempt to load dataset
    report = extract_and_verify_dataset()

    if not report:
        print("\n" + "="*50)
        print("NEXT STEPS:")
        print("1. Download dataset from: http://cvit.iiit.ac.in/research/projects/cvit-projects/the-iiit-5k-word-dataset/")
        print("2. Upload IIIT5K-Word_V3.0.tar to Colab")
        print("   - Click folder icon > 'Upload' in left sidebar")
        print("3. OR use direct download command:")
        print("   !wget http://cvit.iiit.ac.in/research/projects/cvit-projects/the-iiit-5k-word-dataset/IIIT5K-Word_V3.0.tar")
        print("="*50)
        exit(1)

    # Generate dataset loader code
    print("\n" + "="*50)
    print("DATASET LOADER READY - USE THIS IN YOUR OCR CODE")
    print("="*50)
    print(f"# Base directory: {os.path.abspath(report['base_dir'])}")
    print("from torch.utils.data import Dataset\n")
    print("class IIIT5KDataset(Dataset):")
    print("    def __init__(self, root_dir, split='train'):")
    print("        self.root_dir = root_dir")
    print("        self.split = split")
    print("        ")
    print("        # Load ground truth")
    print("        gt_path = r'{}'".format(os.path.abspath(report['gt_file'])))
    print("        # For .mat files: use scipy.io.loadmat(gt_path)")
    print("        # For .txt files: parse as text")
    print("        ")
    print("        # Set image directory")
    print("        self.img_dir = r'{}'".format(
        os.path.abspath(report['train_dir']) if report['split'] == 'train' else os.path.abspath(report['test_dir'])
    ))
    print("        ")
    print("        # Parse ground truth (example for text format)")
    print("        self.samples = []")
    print("        with open(gt_path, 'r') as f:")
    print("            for line in f:")
    print("                parts = line.strip().split()  # Adjust based on actual format")
    print("                if len(parts) >= 2:")
    print("                    img_name = parts[0]")
    print("                    text = ' '.join(parts[1:])")
    print("                    self.samples.append((img_name, text))")
    print("        ")
    print("    def __len__(self):")
    print("        return len(self.samples)")
    print("    ")
    print("    def __getitem__(self, idx):")
    print("        img_name, text = self.samples[idx]")
    print("        img_path = os.path.join(self.img_dir, img_name)")
    print("        image = Image.open(img_path).convert('L')  # Grayscale")
    print("        return image, text")
    print("="*50)

IIIT5K DATASET LOADER - SMART FIX
ERROR: IIIT5K dataset not found!
Please upload the dataset using:
1. Google Drive: Mount Drive and specify path
2. Direct upload: Click 'Upload' in Colab file browser
3. Or download with: !wget http://cvit.iiit.ac.in/research/projects/cvit-projects/the-iiit-5k-word-dataset/IIIT5K-Word_V3.0.tar

NEXT STEPS:
1. Download dataset from: http://cvit.iiit.ac.in/research/projects/cvit-projects/the-iiit-5k-word-dataset/
2. Upload IIIT5K-Word_V3.0.tar to Colab
   - Click folder icon > 'Upload' in left sidebar
3. OR use direct download command:
   !wget http://cvit.iiit.ac.in/research/projects/cvit-projects/the-iiit-5k-word-dataset/IIIT5K-Word_V3.0.tar

DATASET LOADER READY - USE THIS IN YOUR OCR CODE


TypeError: 'NoneType' object is not subscriptable

In [20]:
# deep_ocr_custom_split.py
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
import string
import scipy.io
import random

# ======================
# 1. MODIFIED DATASET CLASS WITH CUSTOM SPLIT
# ======================
class CustomSplitIIIT5KDataset(Dataset):
    def __init__(self, root_dir, split='train', img_height=32, img_width=128,
                 train_samples=4500, test_samples=500, random_seed=42):
        self.root_dir = root_dir
        self.split = split
        self.img_height = img_height
        self.img_width = img_width
        self.iiit5k_dir = os.path.join(root_dir, "IIIT5K")

        # Set random seed for reproducible splits
        random.seed(random_seed)
        np.random.seed(random_seed)

        # ✅ STEP 1: Define vocabulary FIRST (critical fix!)
        self.chars = string.ascii_letters + string.digits + "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
        self.char_to_idx = {char: idx + 1 for idx, char in enumerate(self.chars)}
        self.idx_to_char = {idx + 1: char for idx, char in enumerate(self.chars)}
        self.idx_to_char[0] = '<BLANK>'
        self.num_classes = len(self.chars) + 1  # 95 + 1

        print(f"Creating custom dataset split with {train_samples} train, {test_samples} test samples...")

        # Load all data from both original train and test sets
        all_samples = []

        # Load original train data
        train_mat_file = os.path.join(self.iiit5k_dir, "traindata.mat")
        train_data_dir = os.path.join(self.iiit5k_dir, "train")
        if os.path.exists(train_mat_file):
            all_samples.extend(self._load_mat_data(train_mat_file, train_data_dir, "train"))

        # Load original test data
        test_mat_file = os.path.join(self.iiit5k_dir, "testdata.mat")
        test_data_dir = os.path.join(self.iiit5k_dir, "test")
        if os.path.exists(test_mat_file):
            all_samples.extend(self._load_mat_data(test_mat_file, test_data_dir, "test"))

        print(f"Total samples loaded: {len(all_samples)}")

        # Shuffle all samples for random split
        random.shuffle(all_samples)

        # Create custom train/test split
        total_needed = train_samples + test_samples
        if len(all_samples) < total_needed:
            print(f"⚠️ Warning: Only {len(all_samples)} samples available, but {total_needed} requested")
            train_samples = min(train_samples, len(all_samples) - test_samples)
            test_samples = len(all_samples) - train_samples

        if split == 'train':
            self.samples = all_samples[:train_samples]
            print(f"✅ Created training set with {len(self.samples)} samples")
        else:  # test
            self.samples = all_samples[train_samples:train_samples + test_samples]
            print(f"✅ Created test set with {len(self.samples)} samples")

        if self.samples:
            sample_texts = [t for _, t in self.samples[:10]]
            lengths = [len(t) for _, t in self.samples]
            print(f"Sample texts: {sample_texts}")
            print(f"Text lengths: min={min(lengths)}, max={max(lengths)}, avg={np.mean(lengths):.1f}")
            print(f"Vocabulary size: {self.num_classes}")

        # Image transform
        self.transform = T.Compose([
            T.ToPILImage(),
            T.Resize((img_height, img_width), antialias=True),
            T.Grayscale(),
            T.ToTensor(),
            T.Normalize((0.5,), (0.5,))  # [-1, 1]
        ])

    def _load_mat_data(self, mat_file, data_dir, original_split):
        """Load data from .mat file and return list of (img_path, text) tuples"""
        samples = []

        try:
            mat_data = scipy.io.loadmat(mat_file)
        except Exception as e:
            print(f"❌ Failed to load {mat_file}: {e}")
            return samples

        data_key = f"{original_split}data"
        if data_key not in mat_data:
            available = [k for k in mat_data.keys() if not k.startswith('__')]
            print(f"❌ Key '{data_key}' not found in {mat_file}. Available: {available}")
            return samples

        data = mat_data[data_key]
        N = data.shape[1] if data.shape[0] == 1 else data.shape[0]
        print(f"Processing {N} samples from {original_split} data...")

        for i in range(N):
            try:
                sample = data[0, i] if data.shape[0] == 1 else data[i, 0]

                # Extract image path
                img_name = self._safe_extract_string(sample[0])
                if not img_name:
                    img_filename = f"sample_{i}.png"
                else:
                    img_filename = os.path.basename(img_name.strip())
                img_path = os.path.join(data_dir, img_filename)

                # Extract text
                text = self._safe_extract_string(sample[3])
                text = self._clean_text(text)

                if os.path.exists(img_path) and 1 <= len(text) <= 23:
                    samples.append((img_path, text))

            except Exception as e:
                continue

        return samples

    def _safe_extract_string(self, data):
        """Robustly extract string from .mat field (handles str, numeric array, object array)"""
        try:
            if isinstance(data, str):
                return data.strip()

            if isinstance(data, np.ndarray):
                # Case 1: Numeric array (ASCII codes)
                if np.issubdtype(data.dtype, np.number):
                    chars = []
                    for c in data.flatten():
                        try:
                            c_int = int(c)
                            if 32 <= c_int <= 126:  # Printable ASCII
                                chars.append(chr(c_int))
                        except (ValueError, TypeError):
                            continue
                    return ''.join(chars).strip()

                # Case 2: String array (U/S dtype)
                elif data.dtype.kind in ['U', 'S']:
                    return str(data.flatten()[0]).strip() if data.size > 0 else ""

                # Case 3: Object array (common in scipy.io.loadmat)
                elif data.dtype == np.object_:
                    item = data.item() if data.size == 1 else data[0]
                    if isinstance(item, str):
                        return item.strip()
                    return self._safe_extract_string(item)

            return str(data).strip()
        except Exception as e:
            return ""

    def _clean_text(self, text):
        """Keep only valid characters in full ASCII set"""
        return ''.join(c for c in text if c in self.chars).strip()

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, text = self.samples[idx]
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"⚠️ Warning: Failed to load image {img_path}, using blank")
            img = np.ones((self.img_height, self.img_width), dtype=np.uint8) * 255

        text_indices = [self.char_to_idx[char] for char in text if char in self.char_to_idx]
        img_tensor = self.transform(img)
        return img_tensor, text_indices, text


# ======================
# 2. DEEP CRNN MODEL (UNCHANGED)
# ======================
class DeepCRNN(nn.Module):
    def __init__(self, img_height=32, num_classes=96, hidden_size=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d((2, 1), (2, 1)),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1), (2, 1)),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1), (2, 1)),
        )
        self.rnn = nn.LSTM(
            input_size=512,
            hidden_size=hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=False,
            dropout=0.3
        )
        self.classifier = nn.Linear(hidden_size * 2, num_classes)
        self.dropout = nn.Dropout(0.4)
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        conv = self.cnn(x)  # [B, 512, 1, W]
        b, c, h, w = conv.size()
        conv = conv.view(b, c * h, w).permute(2, 0, 1)  # [W, B, 512]
        rnn_out, _ = self.rnn(conv)
        rnn_out = self.dropout(rnn_out)
        output = self.classifier(rnn_out)
        return F.log_softmax(output, dim=2)


# ======================
# 3. CTC UTILITIES (UNCHANGED)
# ======================
def ctc_collate_fn(batch):
    images, text_indices_list, raw_texts = zip(*batch)
    images = torch.stack(images, 0)
    targets = []
    target_lengths = []
    for text_indices in text_indices_list:
        if len(text_indices) > 0:
            targets.extend(text_indices)
            target_lengths.append(len(text_indices))
        else:
            targets.append(0)
            target_lengths.append(1)
    return images, torch.tensor(targets), torch.tensor(target_lengths), raw_texts

def ctc_decode(log_probs, idx_to_char, blank_idx=0):
    """Greedy CTC decoding"""
    seq_len, batch_size, num_classes = log_probs.shape
    predictions = []
    for b in range(batch_size):
        pred_indices = log_probs[:, b, :].argmax(dim=1).cpu().numpy()
        decoded = []
        prev_idx = blank_idx
        for idx in pred_indices:
            if idx != blank_idx and idx != prev_idx:
                if idx in idx_to_char and idx_to_char[idx] != '<BLANK>':
                    decoded.append(idx_to_char[idx])
            prev_idx = idx
        predictions.append(''.join(decoded))
    return predictions

def calculate_metrics(pred_texts, true_texts):
    """Calculate word and character accuracy"""
    if not pred_texts or not true_texts:
        return 0.0, 0.0
    word_acc = sum(p == t for p, t in zip(pred_texts, true_texts)) / len(true_texts)
    total_chars = sum(max(len(p), len(t)) for p, t in zip(pred_texts, true_texts))
    correct_chars = sum(p[i] == t[i] for p, t in zip(pred_texts, true_texts) for i in range(min(len(p), len(t))))
    char_acc = correct_chars / total_chars if total_chars > 0 else 0.0
    return word_acc, char_acc


# ======================
# 4. TRAINING LOOP WITH CUSTOM SPLIT
# ======================
def train_deep_ocr_custom_split():
    print("🚀 DEEP OCR TRAINING WITH CUSTOM SPLIT (4500 train, 500 test)")
    print("=" * 60)

    # CONFIGURATION - Updated based on your dataset structure
    dataset_root = "/content/iiit5k_dataset"  # Your extracted dataset path

    # Verify dataset exists
    print(f"🔍 Checking dataset root: {dataset_root}")
    print(f"📁 Path exists: {os.path.exists(dataset_root)}")

    if not os.path.exists(dataset_root):
        # Try alternative paths
        alternative_paths = ["./iiit5k_dataset", "/content/iiit5k_dataset"]
        for alt_path in alternative_paths:
            if os.path.exists(alt_path):
                dataset_root = alt_path
                print(f"✅ Found dataset at alternative path: {dataset_root}")
                break
        else:
            print(f"❌ Dataset root not found at any location")
            print("💡 Checked paths:")
            for path in ["/content/iiit5k_dataset"] + alternative_paths:
                print(f"   {path}: {'✅' if os.path.exists(path) else '❌'}")
            return None
    else:
        print(f"✅ Dataset root found: {dataset_root}")

    # Verify IIIT5K subdirectory structure
    iiit5k_path = os.path.join(dataset_root, "IIIT5K")
    print(f"🔍 Checking IIIT5K path: {iiit5k_path}")
    print(f"📁 IIIT5K exists: {os.path.exists(iiit5k_path)}")

    if os.path.exists(iiit5k_path):
        contents = os.listdir(iiit5k_path)
        print(f"📊 IIIT5K contents: {contents}")

        # Check required files
        required_items = ['train', 'test', 'traindata.mat', 'testdata.mat']
        missing_items = [item for item in required_items if item not in contents]

        if missing_items:
            print(f"❌ Missing required items: {missing_items}")
            return None
        else:
            print("✅ All required dataset components found!")
    else:
        print(f"❌ IIIT5K subdirectory not found at: {iiit5k_path}")
        return None

    batch_size = 32
    num_epochs = 250
    learning_rate = 1e-3
    train_samples = 4500
    test_samples = 500

    print(f"Custom split: {train_samples} train, {test_samples} test")
    print(f"Batch size: {batch_size}")
    print(f"Epochs: {num_epochs}")
    print(f"Learning rate: {learning_rate}")

    # Load datasets with custom split
    print("\nCreating custom split datasets...")
    try:
        train_dataset = CustomSplitIIIT5KDataset(
            dataset_root, 'train', 32, 128, train_samples, test_samples
        )
        test_dataset = CustomSplitIIIT5KDataset(
            dataset_root, 'test', 32, 128, train_samples, test_samples
        )
    except Exception as e:
        print(f"❌ Dataset creation failed: {e}")
        return None

    if len(train_dataset) == 0 or len(test_dataset) == 0:
        print("❌ No valid data loaded.")
        return None

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              collate_fn=ctc_collate_fn, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                             collate_fn=ctc_collate_fn, num_workers=0)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\nDevice: {device}")

    model = DeepCRNN(32, train_dataset.num_classes).to(device)
    criterion = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
    optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)

    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Training batches: {len(train_loader)}")
    print(f"Test batches: {len(test_loader)}")

    best_word_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_word_acc = 0.0
        num_train_batches = 0

        print(f"\n📈 Epoch {epoch+1}/{num_epochs}")
        pbar = tqdm(train_loader, desc="Training")
        for images, targets, target_lengths, raw_texts in pbar:
            images = images.to(device)
            targets = targets.to(device)
            target_lengths = target_lengths.to(device)

            log_probs = model(images)
            input_lengths = torch.full((images.size(0),), log_probs.size(0), device=device, dtype=torch.long)
            loss = criterion(log_probs, targets, input_lengths, target_lengths)

            optimizer.zero_grad()
            loss.backward()
            clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss += loss.item()

            # Log every 20 batches (adjusted for potentially more batches)
            if num_train_batches % 20 == 0:
                pred_texts = ctc_decode(log_probs.cpu(), train_dataset.idx_to_char)
                word_acc, _ = calculate_metrics(pred_texts, raw_texts)
                train_word_acc += word_acc
                pbar.set_postfix({"loss": f"{loss.item():.3f}", "word_acc": f"{word_acc:.3f}"})

            num_train_batches += 1

        scheduler.step()
        train_loss /= num_train_batches
        train_word_acc /= (num_train_batches // 20 + 1)

        # Validation
        model.eval()
        val_preds, val_truths = [], []
        with torch.no_grad():
            for images, targets, target_lengths, raw_texts in test_loader:
                images = images.to(device)
                log_probs = model(images)
                pred_texts = ctc_decode(log_probs.cpu(), test_dataset.idx_to_char)
                val_preds.extend(pred_texts)
                val_truths.extend(raw_texts)

        val_word_acc, val_char_acc = calculate_metrics(val_preds, val_truths)

        print(f"\n📊 Epoch {epoch+1} Summary:")
        print(f"  Training Loss: {train_loss:.4f}")
        print(f"  Training Word Acc: {train_word_acc:.4f}")
        print(f"  Validation Word Acc: {val_word_acc:.4f}")
        print(f"  Validation Char Acc: {val_char_acc:.4f}")
        print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")

        if val_word_acc > best_word_acc:
            best_word_acc = val_word_acc
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'val_word_acc': val_word_acc,
                'idx_to_char': train_dataset.idx_to_char,
                'num_classes': train_dataset.num_classes
            }, 'deep_ocr_custom_split_model.pth')
            print(f"  ✅ New best model saved! Word Acc: {best_word_acc:.4f}")

        # Sample predictions
        print(f"\n🎯 Sample predictions:")
        indices = np.random.choice(len(val_preds), min(5, len(val_preds)), replace=False)
        for i in indices:
            print(f"  True: '{val_truths[i]}' | Pred: '{val_preds[i]}'")

    print(f"\n🎉 Training completed!")
    print(f"Best validation word accuracy: {best_word_acc:.4f}")
    return best_word_acc


# ======================
# 5. MAIN EXECUTION
# ======================
if __name__ == "__main__":
    print("⚡ DEEP OCR SYSTEM WITH CUSTOM SPLIT")
    print("=" * 50)
    choice = input("\nChoose option:\n1. Train with custom split (4500 train, 500 test)\n2. Exit\nChoice: ")
    if choice == "1":
        accuracy = train_deep_ocr_custom_split()
        if accuracy is not None:
            print(f"\n🏆 Final result: {accuracy:.4f} word accuracy")
        else:
            print("\n❌ Training failed. Check error messages above.")
    else:
        print("Goodbye! 👋")

⚡ DEEP OCR SYSTEM WITH CUSTOM SPLIT

Choose option:
1. Train with custom split (4500 train, 500 test)
2. Exit
Choice: 1
🚀 DEEP OCR TRAINING WITH CUSTOM SPLIT (4500 train, 500 test)
🔍 Checking dataset root: /content/iiit5k_dataset
📁 Path exists: True
✅ Dataset root found: /content/iiit5k_dataset
🔍 Checking IIIT5K path: /content/iiit5k_dataset/IIIT5K
📁 IIIT5K exists: False
❌ IIIT5K subdirectory not found at: /content/iiit5k_dataset/IIIT5K

❌ Training failed. Check error messages above.


In [21]:
# Test  the model

import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

# ======================
# 1. DEEP CRNN MODEL DEFINITION (MUST BE INCLUDED!)
# ======================
class DeepCRNN(nn.Module):
    def __init__(self, img_height=32, num_classes=96, hidden_size=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d((2, 1), (2, 1)),
            nn.Conv2d(256, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1), (2, 1)),
            nn.Conv2d(512, 512, 3, 1, 1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1), (2, 1)),
        )
        self.rnn = nn.LSTM(
            input_size=512,
            hidden_size=hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=False,
            dropout=0.3
        )
        self.classifier = nn.Linear(hidden_size * 2, num_classes)
        self.dropout = nn.Dropout(0.4)
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        conv = self.cnn(x)  # [B, 512, 1, W]
        b, c, h, w = conv.size()
        conv = conv.view(b, c * h, w).permute(2, 0, 1)  # [W, B, 512]
        rnn_out, _ = self.rnn(conv)
        rnn_out = self.dropout(rnn_out)
        output = self.classifier(rnn_out)
        return F.log_softmax(output, dim=2)

# ======================
# 2. CTC DECODE FUNCTION
# ======================
def ctc_decode(log_probs, idx_to_char, blank_idx=0):
    """Greedy CTC decoding"""
    pred_indices = log_probs.argmax(dim=2).squeeze(1).cpu().numpy()  # [T]
    decoded = []
    prev_idx = blank_idx
    for idx in pred_indices:
        if idx != blank_idx and idx != prev_idx:
            if idx in idx_to_char and idx_to_char[idx] != '<BLANK>':
                decoded.append(idx_to_char[idx])
        prev_idx = idx
    return ''.join(decoded)

# ======================
# 3. INFERENCE SETUP
# ======================
model_path = "/content/deep_ocr_custom_split_model (1).pth"
image_path = "/content/WhatsApp Image 2025-09-04 at 10.02.39 AM.jpeg"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Image transform
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((32, 128), antialias=True),
    T.Grayscale(),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,))
])

# ======================
# 4. LOAD MODEL
# ======================
print("🚀 Loading trained model...")
checkpoint = torch.load(model_path, map_location=device)
num_classes = checkpoint['num_classes']
idx_to_char = checkpoint['idx_to_char']

# Now we can create the model
model = DeepCRNN(img_height=32, num_classes=num_classes, hidden_size=128).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✅ Model loaded successfully with {num_classes} classes.")

# ======================
# 5. LOAD AND PREPROCESS IMAGE
# ======================
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Cannot load image at {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # To 3-channel for ToPILImage
    return transform(img).unsqueeze(0)  # Add batch dim

print(f"🖼️  Preprocessing image: {image_path}")
image_tensor = preprocess_image(image_path).to(device)

# ======================
# 6. RUN INFERENCE
# ======================
print("🔍 Running inference...")
with torch.no_grad():
    log_probs = model(image_tensor)  # [T, 1, num_classes]
    predicted_text = ctc_decode(log_probs, idx_to_char)

# ======================
# 7. OUTPUT RESULT
# ======================
print("\n" + "="*50)
print("🎯 INFERENCE RESULT")
print("="*50)
print(f"Image: {image_path}")
print(f"Predicted Text: '{predicted_text}'")
print("="*50)

🚀 Loading trained model...
✅ Model loaded successfully with 95 classes.
🖼️  Preprocessing image: /content/WhatsApp Image 2025-09-04 at 10.02.39 AM.jpeg
🔍 Running inference...

🎯 INFERENCE RESULT
Image: /content/WhatsApp Image 2025-09-04 at 10.02.39 AM.jpeg
Predicted Text: 'A'
